In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from langchain.chat_models import init_chat_model
from typing import Callable

large_model = init_chat_model("gpt-5")
standard_model = init_chat_model("gpt-5-nano")

@wrap_model_call
def state_based_model(request: ModelRequest, handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:
  """Select model based on the State Conversation length."""
  message_count = len(request.messages)

  if message_count > 10:
    model = large_model
  else:
    model = standard_model

  request = request.override(model=model)

  return handler(request)

In [4]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano",
    middleware=[state_based_model],
    system_prompt="You are roleplaying a real life helpful office intern."
)

In [5]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [
        HumanMessage(content="Did you water the office plant today?")
        ]}
)

print(response["messages"][-1].content)

I didn’t water it today—I can’t physically reach the plant from here. I can help you set up a reminder or add a quick watering log so we don’t forget next time. Want me to create a daily/weekly reminder (and around what time)? If you’re in the office now and want to do it, I can guide you through a quick check: insert a finger about an inch into the soil; if it’s dry, water until you see a little drainage; don’t leave the plant sitting in standing water.


In [6]:

print(response["messages"][-1].response_metadata["model_name"])

gpt-5-nano-2025-08-07


In [7]:
from langchain.messages import AIMessage

response = agent.invoke(
    {"messages": [
        HumanMessage(content="Did you water the office plant today?"),
        AIMessage(content="Yes, I gave it a light watering this morning."),
        HumanMessage(content="Has it grown much this week?"),
        AIMessage(content="It's sprouted two new leaves since Monday."),
        HumanMessage(content="Are the leaves still turning yellow on the edges?"),
        AIMessage(content="A little, but it's looking healthier overall."),
        HumanMessage(content="Did you remember to rotate the pot toward the window?"),
        AIMessage(content="I rotated it a quarter turn so it gets more even light."),
        HumanMessage(content="How often should we be fertilizing this plant?"),
        AIMessage(content="About once every two weeks with a diluted liquid fertilizer."),
        HumanMessage(content="When should we expect to have to replace the pot?")
        ]}
)

print(response["messages"][-1].content)

Most office plants need a pot upgrade about every 12–24 months, ideally in late winter or spring. Sooner if you see signs it’s rootbound:
- Roots coming out of the drainage holes or circling the surface
- Water racing through the pot or the plant drying out much faster than before
- Slowed growth or the plant getting top‑heavy

When we do repot, we’ll go just 1–2 inches wider with fresh potting mix and good drainage. I can check the roots at the next watering and pencil in repotting for early spring if it looks tight.


In [8]:
print(response["messages"][-1].response_metadata["model_name"])

gpt-5-2025-08-07
